In [1]:
from __future__ import print_function, division
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local") \
   .appName("test") \
   .enableHiveSupport() \
   .getOrCreate()

sc = spark.sparkContext

# 4.4 操作與觀察 Dataframe

## Part1. 基本操作與觀察

### Read file

In [71]:
fileDF = spark.read.csv("hdfs:///tmp/ratings.csv", sep = ',', header = True)

### 觀察檔案

In [72]:
fileDF.show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   7155|   3.5|1164885564|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     3|  33750|   3.5|1164885688|
|     3|  33750|   3.5|1164887688|
|     3|    344|  null| 844416742|
|     4|     21|     3| 844416980|
|     4|     34|     5| 844416936|
|     4|     39|     3| 844417037|
|     4|    110|     5| 844416866|
|     4|    150|     5| 844416656|
|     4|    153|     5| 844416699|
|     4|    161|     5| 844416835|
|     4|    165|     5| 844416699|
|     4|    208|     3| 844416866|
|     4|    231|     1| 844416742|
+------+-------+------+----------+
only showing top 20 rows



### 觀察欄位

In [73]:
fileDF.columns

['userid', 'movieid', 'rating', 'ts']

In [74]:
len(fileDF.columns)

4

### 欄位統計值

In [75]:
fileDF.describe().show()

+-------+------------------+-----------------+------------------+-------------------+
|summary|            userid|          movieid|            rating|                 ts|
+-------+------------------+-----------------+------------------+-------------------+
|  count|                29|               28|                27|                 28|
|   mean|3.6551724137931036|5217.678571428572| 4.018518518518518|9.410967772142857E8|
| stddev|0.4837252813149749|9923.353021491952|1.1135656648929557|1.432772681721757E8|
|    min|                 3|              110|                 1|         1133571171|
|    max|                 4|             8783|                 5|          844417070|
+-------+------------------+-----------------+------------------+-------------------+



In [76]:
fileDF.describe('userid').show()

+-------+------------------+
|summary|            userid|
+-------+------------------+
|  count|                29|
|   mean|3.6551724137931036|
| stddev|0.4837252813149749|
|    min|                 3|
|    max|                 4|
+-------+------------------+



### 打印 schema

In [77]:
fileDF.printSchema()

root
 |-- userid: string (nullable = true)
 |-- movieid: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- ts: string (nullable = true)



### 選擇欄位

In [78]:
fileDF.select('userid', 'rating').show()

+------+------+
|userid|rating|
+------+------+
|     3|     5|
|     3|     4|
|     3|   3.5|
|     3|     4|
|     3|   4.5|
|     3|     5|
|     3|   4.5|
|     3|   3.5|
|     3|   3.5|
|     3|  null|
|     4|     3|
|     4|     5|
|     4|     3|
|     4|     5|
|     4|     5|
|     4|     5|
|     4|     5|
|     4|     5|
|     4|     3|
|     4|     1|
+------+------+
only showing top 20 rows



In [79]:
fileDF.select('userid', fileDF['rating'] + 1).show()

+------+------------+
|userid|(rating + 1)|
+------+------------+
|     3|         6.0|
|     3|         5.0|
|     3|         4.5|
|     3|         5.0|
|     3|         5.5|
|     3|         6.0|
|     3|         5.5|
|     3|         4.5|
|     3|         4.5|
|     3|        null|
|     4|         4.0|
|     4|         6.0|
|     4|         4.0|
|     4|         6.0|
|     4|         6.0|
|     4|         6.0|
|     4|         6.0|
|     4|         6.0|
|     4|         4.0|
|     4|         2.0|
+------+------------+
only showing top 20 rows



In [80]:
fileDF.select('userid', fileDF['rating'] + 1).printSchema()

root
 |-- userid: string (nullable = true)
 |-- (rating + 1): double (nullable = true)



### 篩選欄位

In [81]:
fileDF.filter(fileDF['userid'] == 3).show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   7155|   3.5|1164885564|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     3|  33750|   3.5|1164885688|
|     3|  33750|   3.5|1164887688|
|     3|    344|  null| 844416742|
+------+-------+------+----------+



In [82]:
fileDF.filter(fileDF['userid'] == 3).select('userid', 'rating').show()

+------+------+
|userid|rating|
+------+------+
|     3|     5|
|     3|     4|
|     3|   3.5|
|     3|     4|
|     3|   4.5|
|     3|     5|
|     3|   4.5|
|     3|   3.5|
|     3|   3.5|
|     3|  null|
+------+------+



### 計算不重複值

In [83]:
fileDF.count()

29

In [84]:
fileDF.select('userid').distinct().show()

+------+
|userid|
+------+
|     3|
|     4|
+------+



### 小練習: 有幾部電影被評為5分?

In [85]:
fileDF.show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   7155|   3.5|1164885564|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     3|  33750|   3.5|1164885688|
|     3|  33750|   3.5|1164887688|
|     3|    344|  null| 844416742|
|     4|     21|     3| 844416980|
|     4|     34|     5| 844416936|
|     4|     39|     3| 844417037|
|     4|    110|     5| 844416866|
|     4|    150|     5| 844416656|
|     4|    153|     5| 844416699|
|     4|    161|     5| 844416835|
|     4|    165|     5| 844416699|
|     4|    208|     3| 844416866|
|     4|    231|     1| 844416742|
+------+-------+------+----------+
only showing top 20 rows



In [86]:
fileDF.filter(fileDF['rating']==5).select('movieid').distinct().count()

12

## Part2 資料清理

### 觀察數值分配

In [112]:
fileDF.crosstab('userid', 'rating').show()

+-------------+---+---+---+---+---+---+---+----+
|userid_rating|  1|  2|  3|3.5|  4|4.5|  5|null|
+-------------+---+---+---+---+---+---+---+----+
|            4|  1|  1|  6|  0|  0|  0| 10|   1|
|            3|  0|  0|  0|  3|  2|  2|  2|   1|
+-------------+---+---+---+---+---+---+---+----+



### 處理遺漏值

In [109]:
fileDF.fillna(0).show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   7155|   3.5|1164885564|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     3|  33750|   3.5|1164885688|
|     3|  33750|   3.5|1164887688|
|     3|    344|  null| 844416742|
|     4|     21|     3| 844416980|
|     4|     34|     5| 844416936|
|     4|     39|     3| 844417037|
|     4|    110|     5| 844416866|
|     4|    150|     5| 844416656|
|     4|    153|     5| 844416699|
|     4|    161|     5| 844416835|
|     4|    165|     5| 844416699|
|     4|    208|     3| 844416866|
|     4|    231|     1| 844416742|
+------+-------+------+----------+
only showing top 20 rows



In [111]:
fileDF.fillna('0').show()

+------+-------+------+----------+
|userid|movieid|rating|        ts|
+------+-------+------+----------+
|     3|   6539|     5|1133571238|
|     3|   7153|     4|1133571171|
|     3|   7155|   3.5|1164885564|
|     3|   8529|     4|1136075616|
|     3|   8533|   4.5|1136418593|
|     3|   8783|     5|1136075857|
|     3|  27821|   4.5|1136418616|
|     3|  33750|   3.5|1164885688|
|     3|  33750|   3.5|1164887688|
|     3|    344|     0| 844416742|
|     4|     21|     3| 844416980|
|     4|     34|     5| 844416936|
|     4|     39|     3| 844417037|
|     4|    110|     5| 844416866|
|     4|    150|     5| 844416656|
|     4|    153|     5| 844416699|
|     4|    161|     5| 844416835|
|     4|    165|     5| 844416699|
|     4|    208|     3| 844416866|
|     4|    231|     1| 844416742|
+------+-------+------+----------+
only showing top 20 rows



### 轉換欄位屬性

In [114]:
from pyspark.sql.types import DoubleType

In [115]:
fileDF = fileDF.withColumn("rating_double", fileDF["rating"].cast(DoubleType()))

In [117]:
fileDF.printSchema()

root
 |-- userid: string (nullable = true)
 |-- movieid: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- rating_double: double (nullable = true)



In [119]:
fileDF.show()

+------+-------+------+----------+-------------+
|userid|movieid|rating|        ts|rating_double|
+------+-------+------+----------+-------------+
|     3|   6539|     5|1133571238|          5.0|
|     3|   7153|     4|1133571171|          4.0|
|     3|   7155|   3.5|1164885564|          3.5|
|     3|   8529|     4|1136075616|          4.0|
|     3|   8533|   4.5|1136418593|          4.5|
|     3|   8783|     5|1136075857|          5.0|
|     3|  27821|   4.5|1136418616|          4.5|
|     3|  33750|   3.5|1164885688|          3.5|
|     3|  33750|   3.5|1164887688|          3.5|
|     3|    344|  null| 844416742|         null|
|     4|     21|     3| 844416980|          3.0|
|     4|     34|     5| 844416936|          5.0|
|     4|     39|     3| 844417037|          3.0|
|     4|    110|     5| 844416866|          5.0|
|     4|    150|     5| 844416656|          5.0|
|     4|    153|     5| 844416699|          5.0|
|     4|    161|     5| 844416835|          5.0|
|     4|    165|    

In [122]:
fileDF.fillna(0).show()

+------+-------+------+----------+-------------+
|userid|movieid|rating|        ts|rating_double|
+------+-------+------+----------+-------------+
|     3|   6539|     5|1133571238|          5.0|
|     3|   7153|     4|1133571171|          4.0|
|     3|   7155|   3.5|1164885564|          3.5|
|     3|   8529|     4|1136075616|          4.0|
|     3|   8533|   4.5|1136418593|          4.5|
|     3|   8783|     5|1136075857|          5.0|
|     3|  27821|   4.5|1136418616|          4.5|
|     3|  33750|   3.5|1164885688|          3.5|
|     3|  33750|   3.5|1164887688|          3.5|
|     3|    344|  null| 844416742|          0.0|
|     4|     21|     3| 844416980|          3.0|
|     4|     34|     5| 844416936|          5.0|
|     4|     39|     3| 844417037|          3.0|
|     4|    110|     5| 844416866|          5.0|
|     4|    150|     5| 844416656|          5.0|
|     4|    153|     5| 844416699|          5.0|
|     4|    161|     5| 844416835|          5.0|
|     4|    165|    

In [124]:
fileDF_clean = fileDF.fillna(0)

In [126]:
fileDF_clean.crosstab('userid', 'rating_double').show()

+--------------------+---+---+---+---+---+---+---+---+
|userid_rating_double|0.0|1.0|2.0|3.0|3.5|4.0|4.5|5.0|
+--------------------+---+---+---+---+---+---+---+---+
|                   4|  1|  1|  1|  6|  0|  0|  0| 10|
|                   3|  1|  0|  0|  0|  3|  2|  2|  2|
+--------------------+---+---+---+---+---+---+---+---+



In [127]:
fileDF.dropna().show()

+------+-------+------+----------+-------------+
|userid|movieid|rating|        ts|rating_double|
+------+-------+------+----------+-------------+
|     3|   6539|     5|1133571238|          5.0|
|     3|   7153|     4|1133571171|          4.0|
|     3|   7155|   3.5|1164885564|          3.5|
|     3|   8529|     4|1136075616|          4.0|
|     3|   8533|   4.5|1136418593|          4.5|
|     3|   8783|     5|1136075857|          5.0|
|     3|  27821|   4.5|1136418616|          4.5|
|     3|  33750|   3.5|1164885688|          3.5|
|     3|  33750|   3.5|1164887688|          3.5|
|     4|     21|     3| 844416980|          3.0|
|     4|     34|     5| 844416936|          5.0|
|     4|     39|     3| 844417037|          3.0|
|     4|    110|     5| 844416866|          5.0|
|     4|    150|     5| 844416656|          5.0|
|     4|    153|     5| 844416699|          5.0|
|     4|    161|     5| 844416835|          5.0|
|     4|    165|     5| 844416699|          5.0|
|     4|    208|    

### 處理重復值

In [151]:
fileDF.show()

+------+-------+------+----------+-------------+
|userid|movieid|rating|        ts|rating_double|
+------+-------+------+----------+-------------+
|     3|   6539|     5|1133571238|          5.0|
|     3|   7153|     4|1133571171|          4.0|
|     3|   7155|   3.5|1164885564|          3.5|
|     3|   8529|     4|1136075616|          4.0|
|     3|   8533|   4.5|1136418593|          4.5|
|     3|   8783|     5|1136075857|          5.0|
|     3|  27821|   4.5|1136418616|          4.5|
|     3|  33750|   3.5|1164885688|          3.5|
|     3|  33750|   3.5|1164887688|          3.5|
|     3|    344|  null| 844416742|         null|
|     4|     21|     3| 844416980|          3.0|
|     4|     34|     5| 844416936|          5.0|
|     4|     39|     3| 844417037|          3.0|
|     4|    110|     5| 844416866|          5.0|
|     4|    150|     5| 844416656|          5.0|
|     4|    153|     5| 844416699|          5.0|
|     4|    161|     5| 844416835|          5.0|
|     4|    165|    

In [156]:
fileDF.crosstab("userid", "movieid").show()

+--------------+---+---+---+---+---+---+---+---+---+---+-----+---+---+---+---+-----+---+---+---+---+----+----+----+----+----+----+----+
|userid_movieid|110|150|153|161|165|208| 21|231|253|266|27821|292|316|317|329|33750| 34|344|349| 39|6539|7153|7155|8529|8533|8783|null|
+--------------+---+---+---+---+---+---+---+---+---+---+-----+---+---+---+---+-----+---+---+---+---+----+----+----+----+----+----+----+
|             4|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|    0|  1|  1|  1|  1|    0|  1|  1|  1|  1|   0|   0|   0|   0|   0|   0|   1|
|             3|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|    1|  0|  0|  0|  0|    2|  0|  1|  0|  0|   1|   1|   1|   1|   1|   1|   0|
+--------------+---+---+---+---+---+---+---+---+---+---+-----+---+---+---+---+-----+---+---+---+---+----+----+----+----+----+----+----+



In [160]:
(fileDF.select("userid", "movieid", "rating").count() - 
    fileDF.select("userid", "movieid", "rating").distinct().count())

1

In [149]:
fileDF.dropDuplicates().orderBy(['userid', 'movieid', 'rating'], ascending=[1,0]).show()

+------+-------+------+----------+-------------+
|userid|movieid|rating|        ts|rating_double|
+------+-------+------+----------+-------------+
|     3|   8783|     5|1136075857|          5.0|
|     3|   8533|   4.5|1136418593|          4.5|
|     3|   8529|     4|1136075616|          4.0|
|     3|   7155|   3.5|1164885564|          3.5|
|     3|   7153|     4|1133571171|          4.0|
|     3|   6539|     5|1133571238|          5.0|
|     3|    344|  null| 844416742|         null|
|     3|  33750|   3.5|1164887688|          3.5|
|     3|  33750|   3.5|1164885688|          3.5|
|     3|  27821|   4.5|1136418616|          4.5|
|     4|     39|     3| 844417037|          3.0|
|     4|    349|     3| 844416699|          3.0|
|     4|    344|     2| 844416699|          2.0|
|     4|     34|     5| 844416936|          5.0|
|     4|    329|     5| 844416796|          5.0|
|     4|    317|     5| 844417037|          5.0|
|     4|    316|     5| 844416742|          5.0|
|     4|    292|    

In [150]:
fileDF.dropDuplicates(['userid', 'movieid', 'rating']).orderBy(['userid', 'movieid', 'rating'], ascending=[1,0]).show()

+------+-------+------+----------+-------------+
|userid|movieid|rating|        ts|rating_double|
+------+-------+------+----------+-------------+
|     3|   8783|     5|1136075857|          5.0|
|     3|   8533|   4.5|1136418593|          4.5|
|     3|   8529|     4|1136075616|          4.0|
|     3|   7155|   3.5|1164885564|          3.5|
|     3|   7153|     4|1133571171|          4.0|
|     3|   6539|     5|1133571238|          5.0|
|     3|    344|  null| 844416742|         null|
|     3|  33750|   3.5|1164885688|          3.5|
|     3|  27821|   4.5|1136418616|          4.5|
|     4|     39|     3| 844417037|          3.0|
|     4|    349|     3| 844416699|          3.0|
|     4|    344|     2| 844416699|          2.0|
|     4|     34|     5| 844416936|          5.0|
|     4|    329|     5| 844416796|          5.0|
|     4|    317|     5| 844417037|          5.0|
|     4|    316|     5| 844416742|          5.0|
|     4|    292|     3| 844416796|          3.0|
|     4|    266|    

In [161]:
fileDF_nodup = fileDF.dropDuplicates(['userid', 'movieid', 'rating']).orderBy(['userid', 'movieid', 'rating'], ascending=[1,0])

In [162]:
(fileDF_nodup.select("userid", "movieid", "rating").count() - 
    fileDF_nodup.select("userid", "movieid", "rating").distinct().count())

0

In [163]:
spark.stop()